<a href="https://colab.research.google.com/github/ndronen/econdata/blob/main/notebooks/Electricity_consumption_and_GDP_growth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
#@title Imports
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive

In [2]:
#@title Mount Google Drive for saving plots
DRIVE_PATH = "/content/gdrive"
NOTEBOOK_PATH = f"{DRIVE_PATH}/My Drive/Colab Notebooks"
drive.mount(DRIVE_PATH)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
#@title Set Pandas options
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)
pd.set_option('mode.chained_assignment', 'raise')

In [31]:
#@title Data access functions
GDP_COUNTRY_CODE = "Country Code"
GDP_COUNTRY_NAME = "Country Name"
ELECTRICITY_COUNTRY_CODE = "Country Code"
ELECTRICITY_COUNTRY_NAME = "Country Name"


def get_years_from_column_names(df):
    years = []
    for column in df.columns:
        if re.match(r"^\d\d\d\d$", column):
            years.append(column)
    return years
            

def get_gdp_country_codes(gdp_df):
    codes = gdp_df[GDP_COUNTRY_CODE]
    return codes


def get_electricity_country_codes(electricity_df):
    codes = electricity_df[ELECTRICITY_COUNTRY_CODE]
    return codes


def get_common_codes(gdp_codes, electricity_codes):
    common_codes = set(gdp_codes).intersection(electricity_codes)
    return common_codes


def get_common_code_subset(common_codes, df, code_column_name):
    mask = df[code_column_name].apply(lambda code: code in common_codes)
    df = df[mask]
    df = df.sort_values(code_column_name)
    return df


def join_dataframes(gdp_df, electricity_df):
    # Rename the year columns of the electricity data frame.
    years = get_years_from_column_names(electricity_df)
    year_kwhs = []
    for year in years:
        year_kwh = year + "kwh"
        electricity_df[year_kwh] = electricity_df[year]
        year_kwhs.append(year_kwh)
    electricity_df.drop(labels=years, axis=1, inplace=True)

    years = get_years_from_column_names(gdp_df)
    non_year_columns = [
        column for column in gdp_df.columns if column not in years
    ]
    gdp_df.reset_index(inplace=True)
    electricity_df.reset_index(inplace=True)
    joined_df = pd.concat((gdp_df, electricity_df[year_kwhs]), axis=1)
    return joined_df


def only_countries_with_gdp_in_year(df, year):
    has_gdp = ~df[year].isnull()
    return df[has_gdp]


def get_extreme_countries_by_region(df, sort_column, k=1, method='head'):
    if method == 'head':
        def _sort_values(grp):
            return grp.sort_values(sort_column).head(k)
    else:
        def _sort_values(grp):
            return grp.sort_values(sort_column).tail(k)

    extremes = []
    for region, grp in df.groupby("Region"):
        extremes.append(_sort_values(grp))
    df = pd.concat(extremes)
    return df

In [37]:
#@title Plotting functions

# def plot_cpi_against_change_in_gdp_per_capita(
#     df, start_year, end_year, xmax=None, annotate_extremes=0
# ):
#     def get_codes(year):
#         return only_countries_with_gdp_in_year(df, year)[GDP_COUNTRY_CODE]

#     start_year_codes = get_codes(start_year)
#     end_year_codes = get_codes(end_year)
#     common_codes = get_common_codes(start_year_codes, end_year_codes)

#     mask = df[GDP_COUNTRY_CODE].apply(lambda code: code in common_codes)
#     df_start_end = df[mask].copy()
#     change_in_gdp_per_capita = df_start_end[end_year] - df_start_end[start_year]
#     year_range_column = f"{start_year}-{end_year}"
#     df_start_end.loc[:, year_range_column] = change_in_gdp_per_capita

#     fig, ax = plt.subplots(1, figsize=(10, 6))
#     cmap = plt.get_cmap("tab20")
#     markers = ["+", "D", "o", "s", ">", "X"]
#     for i, (region, grp) in enumerate(df_start_end.groupby("Region")):
#         ax.scatter(
#             grp[year_range_column],
#             grp[ELECTRICITY_INDEX],
#             label=region,
#             color=cmap(i),
#             marker=markers[i]
#         )

#     if annotate_extremes > 0:
#         def _annotate_extremes(df):
#             for code, grp in df.groupby(GDP_COUNTRY_CODE):
#                 x = grp[year_range_column].values[0]
#                 y = grp[ELECTRICITY_INDEX].values[0]
#                 ax.text(x + x*.01, y + y*.01, grp[GDP_COUNTRY_NAME].values[0])

#         # Label countries with greatest or least growth in GDP per capita.
#         extreme_countries = set()
#         for method in {'head', 'tail'}:
#             for column_name in {year_range_column, ELECTRICITY_INDEX}:
#                 df_extreme = get_extreme_countries_by_region(
#                     df_start_end, column_name, k=annotate_extremes, method=method
#                 )
#                 extreme_countries.update(df_extreme[GDP_COUNTRY_CODE].tolist())

#         def is_extreme(code):
#             return code in extreme_countries

#         extreme_country_rows = df_start_end[GDP_COUNTRY_CODE].apply(is_extreme)
#         df_extremes = df_start_end[extreme_country_rows]
#         _annotate_extremes(df_extremes)
            
#     ax.set_xlabel(f"Change in GDP per capita from {start_year} to {end_year}")
#     ax.set_ylabel("Electricity (kWh per capita) 2021")
#     ax.legend(title="Region", loc='lower right')

#     if xmax is not None:
#         ax.set_xlim((0, xmax))

#     ylim = ax.get_ylim()
#     ax.set_ylim((ylim[0], 100))

#     fig.savefig(
#         f"{NOTEBOOK_PATH}/change-in-gdp-per-capita-{start_year}-to-{end_year}.png",
#         dpi=300
#     )

#     return df_start_end, year_range_column

In [33]:
#@title Load data
base_url = "https://raw.githubusercontent.com/ndronen/econdata/6ed22721163d512788510bdc57b688cc7cd0b80a/data/"
electricity_url = f"{base_url}/kwh-power-consumption-per-capita-world-bank.csv"
gdp_url = f"{base_url}/gdp-per-capita-of-countries-in-current-usd-world-bank.csv"
electricity_df = pd.read_csv(electricity_url)
gdp_df = pd.read_csv(gdp_url)

In [38]:
from dataclasses import dataclass

@dataclass
class WorldBankData:
    """Class for keeping track of an item in inventory."""
    gdp: pd.DataFrame
    electricity: pd.DataFrame

In [39]:
#@title Preprocess and join the GDP and electricity data.
gdp_country_codes = get_gdp_country_codes(gdp_df)
electricity_country_codes = get_electricity_country_codes(electricity_df)
common_codes = get_common_codes(gdp_country_codes, electricity_country_codes)
clean_gdp_df = get_common_code_subset(common_codes, gdp_df, GDP_COUNTRY_CODE)
clean_electricity_df = get_common_code_subset(common_codes, electricity_df, ELECTRICITY_COUNTRY_CODE)
world_bank_data = WorldBankData(clean_gdp_df, clean_electricity_df)

In [89]:
def plot_electricity_consumption_for_countries(df, countries):
    years = np.array(get_years_from_column_names(df))
    cmap = plt.get_cmap("tab20")

    fig, ax = plt.subplots(1, figsize=(12, 4))
    max_year = int(years[0])
    for i, country in enumerate(countries):
        df_country = df[df[GDP_COUNTRY_NAME] == country]        
        recorded_years = ~df_country[years].isnull().values[0]
        if len(years[recorded_years]):
            x = [int(ry) for ry in years[recorded_years]]
            if max(x) > max_year:
                max_year = max(x)
            y = df_country[years].values[0][recorded_years]
            ax.plot(x, y, label=country, c=cmap(i))
    years = years.astype(int)
    ax.set_xticks(years)
    ax.set_xticklabels(ax.get_xticks(), rotation=75)
    ax.set_xlim((years[0], max_year + 1))
    ax.set_xlabel("Year")
    ax.set_ylabel("Electricity consumption (kWh) per capita")
    ax.grid()
    plt.legend()

In [90]:
largest_by_gdp = {
    "United States",
    "China",
    "Japan",
    "Germany",
    "United Kingdom",
    "France",
    "Italy"
}

plot_electricity_consumption_for_countries(
    world_bank_data.electricity,
    largest_by_gdp
)